In [23]:
# Setup

import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import glob

csv_dir = os.path.join("..", "..", "data/out_csvs")
hbond_heatmap_csv_dir = os.path.join(csv_dir, "heatmap_data")

def __safe_mkdir(directory: str) -> None:
    """Safely creates a directory if it does not already exist."""
    os.makedirs(directory, exist_ok=True)


def save_present_hbonds(grouped_hbond_df: pd.core.groupby.DataFrameGroupBy) -> None:
    """
    Saves groups of H-bonding data into CSVs.

    Args:
        grouped_hbond_df (pd.core.groupby.DataFrameGroupBy): dataframe containing H-bonding data

    Returns:
        None

    """
    for group_name, hbonds in grouped_hbond_df:
        (res_1, atom_1, res_2, atom_2) = group_name
        type_1, type_2 = res_1, res_2

        print(f"Processing {type_1}-{type_2} {atom_1}-{atom_2}")
        map_name = f"{type_1}-{type_2} {atom_1}-{atom_2}"

        heatmap_csv_path = "data/out_csvs/heatmap_data"
        os.makedirs(heatmap_csv_path, exist_ok=True)

        heat_data_csv_path = f"{heatmap_csv_path}/{map_name}.csv"
        hbonds.to_csv(heat_data_csv_path, index=False)


In [24]:
# Take interactions_detailed, group, then run through figure_plotting function.

# Read the CSV file into a DataFrame
full_interaction_df = pd.read_csv(os.path.join(csv_dir, "interactions_detailed.csv"))

# Normalize the pairs to ensure that reverse orders fall into the same group
full_interaction_df['normalized_pair'] = full_interaction_df.apply(
    lambda row: tuple(sorted([(row['mol_1'], row['atom_1']), (row['mol_2'], row['atom_2'])])),
    axis=1
)

# Drop unnecessary columns and group by normalized pairs
full_interaction_df['mol_1'], full_interaction_df['atom_1'] = zip(*full_interaction_df['normalized_pair'].apply(lambda x: x[0]))
full_interaction_df['mol_2'], full_interaction_df['atom_2'] = zip(*full_interaction_df['normalized_pair'].apply(lambda x: x[1]))

# Drop the intermediate column as it's no longer needed
full_interaction_df.drop(columns=['normalized_pair'], inplace=True)

# Group by the mol_1, atom_1, mol_2, atom_2 pairs
grouped_df = full_interaction_df.groupby(['mol_1', 'atom_1', 'mol_2', 'atom_2'])

# Now pass the grouped object to your save_present_hbonds function
save_present_hbonds(grouped_df)





Processing A-A N1-N1
Processing A-A N1-N3
Processing A-A N1-N6
Processing A-A N1-N7
Processing A-A N1-O2'
Processing A-A N1-O3'
Processing A-A N1-O4'
Processing A-A N1-O5'
Processing A-A N1-OP1
Processing A-A N1-OP2
Processing A-ALA N1-N
Processing A-ALA N1-O
Processing A-ARG N1-N
Processing A-ARG N1-NE
Processing A-ARG N1-NH1
Processing A-ARG N1-NH2
Processing A-ASN N1-N
Processing A-ASN N1-ND2
Processing A-ASN N1-OD1
Processing A-ASP N1-N
Processing A-C N1-N3
Processing A-C N1-N4
Processing A-C N1-O2
Processing A-C N1-O2'
Processing A-C N1-O3'
Processing A-C N1-O4'
Processing A-C N1-O5'
Processing A-C N1-OP1
Processing A-C N1-OP2
Processing A-DT N1-N3
Processing A-G N1-N1
Processing A-G N1-N2
Processing A-G N1-N3
Processing A-G N1-N7
Processing A-G N1-O2'
Processing A-G N1-O3'
Processing A-G N1-O4'
Processing A-G N1-O6
Processing A-G N1-OP1
Processing A-G N1-OP2
Processing A-GLN N1-NE2
Processing A-GLN N1-OE1
Processing A-GLU N1-O
Processing A-GLU N1-OE2
Processing A-GLY N1-N
Process

In [25]:
# Now, for all the H-bond CSVs present,
# make heatmaps showing the relationship
# between the distance and angle

heatmap_csv_files = glob.glob(f"{hbond_heatmap_csv_dir}/*.csv")

for heatmap_csv_file in heatmap_csv_files:
    with open(heatmap_csv_file, 'r') as file:
        # Check the number of lines in the file
        if sum(1 for line in file) < 101:
            continue  # Skip files with fewer than 101 lines
            # i.e. 100 data points
    # Read into DataFrame
    csv_df = pd.read_csv(heatmap_csv_file)
    # Get the name of the CSV file
    heatmap_csv_name = heatmap_csv_file.split("/")[-1]
    heatmap_csv_res_atoms = heatmap_csv_name.split(".")[0]
    # Get the residues and atoms
    heatmap_csv_res = heatmap_csv_res_atoms.split(" ")[0]
    heatmap_csv_atoms = heatmap_csv_res_atoms.split(" ")[1]
    # Get the individual residues
    heatmap_csv_res_1 = heatmap_csv_res.split("-")[0]
    heatmap_csv_res_2 = heatmap_csv_res.split("-")[1]
    # Get the individual atoms
    heatmap_csv_atom_1 = heatmap_csv_atoms.split("-")[0]
    heatmap_csv_atom_2 = heatmap_csv_atoms.split("-")[1]

    # Select distance and angle
    hbonds_subset = csv_df[["distance", "angle"]].reset_index(drop=True)

    # Binning
    distance_bins = np.arange(2.0, 4.1, 0.1)  # Bins from 2 to 4 in increments of 0.1
    angle_bins = np.arange(0, 181, 10)  # Bins from 0 to 180 in increments of 10
    hbonds_subset["distance_bin"] = pd.cut(hbonds_subset["distance"], bins=distance_bins)
    hbonds_subset["angle_bin"] = pd.cut(hbonds_subset["angle"], bins=angle_bins)
    # Data
    heatmap_data = (hbonds_subset.groupby(["angle_bin", "distance_bin"]).size().unstack(fill_value=0))
    # Plot
    plt.figure(figsize=(6, 6))
    sns.heatmap(heatmap_data, cmap="gray_r", xticklabels=1, yticklabels=range(0, 181, 10), square=True)
    plt.xticks(np.arange(len(distance_bins)) + 0.5,[f"{bin_val:.1f}" for bin_val in distance_bins],rotation=0)
    plt.yticks(np.arange(len(angle_bins)) + 0.5, angle_bins, rotation=0)
    plt.xlabel("Distance (Å)")
    plt.ylabel("Angle (°)")
    # Set name and dir
    map_name = f"{heatmap_csv_res_1}-{heatmap_csv_res_2} {heatmap_csv_atom_1}-{heatmap_csv_atom_2}"
    map_dir = (
        "figure_4_heatmaps/RNA-RNA" if len(heatmap_csv_res_1) == 1 and len(heatmap_csv_res_2) == 1
        else "figure_4_heatmaps/RNA-PROT"
    )
    __safe_mkdir(map_dir)
    map_path = f"{map_dir}/{map_name}.png"
    # 2D histogram
    plt.figure(figsize=(6, 4.8))
    plt.hist2d(
        hbonds_subset["distance"],
        hbonds_subset["angle"],
        bins=[distance_bins, angle_bins],
        cmap="gray_r",
    )
    plt.xlabel("Distance (Å)")
    plt.ylabel("Angle (°)")
    plt.colorbar(label="Count")
    plt.title(f"{map_name} H-bond heatmap")

    plt.savefig(map_path, dpi=250)
    plt.close()

plt.close('all')



